<a href="https://colab.research.google.com/github/kgoel99/Mini-Project/blob/main/Mini_Project(KG).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bing-image-downloader

#######################################################################################################

**GATHER IMAGES**

In [2]:
cd /content

/content


In [4]:
!mkdir Mini_P

In [5]:
import bing_image_downloader
from  bing_image_downloader import downloader
downloader.download('acoustic guitar', limit=100,  output_dir='Mini_P', adult_filter_off=True, force_replace=False, timeout=60)
downloader.download('electric guitar', limit=100,  output_dir='Mini_P', adult_filter_off=True, force_replace=False, timeout=60)



[!!]Indexing page: 1

[%] Indexed 100 Images on Page 1.


[%] Downloading Image #1 from https://upload.wikimedia.org/wikipedia/commons/4/4d/C.F._Martin_GRH_160_or_000-16RGT_cropped.png
[%] File Downloaded !

[%] Downloading Image #2 from https://c1.zzounds.com/media/fit,2018by3200/quality,85/1_Full_Straight_Front_NA-52ed1fbb5b59969ca7ae3008d4260979.jpg
[%] File Downloaded !

[%] Downloading Image #3 from http://wallpapercave.com/wp/5VZjLWg.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://c1.zzounds.com/media/productmedia/fit,2018by3200/quality,85/1_Full_Straight_Front_NA-82aaa8a27485a2f40337e33e518e8f52.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://wallpapercave.com/wp/Tea9RiQ.jpg
[%] File Downloaded !

[%] Downloading Image #6 from https://upload.wikimedia.org/wikipedia/commons/thumb/b/b8/Gibson_ES-335_sunburst.jpg/1200px-Gibson_ES-335_sunburst.jpg
[%] File Downloaded !

[%] Downloading Image #7 from https://www.instrumentrentalbarcelona.com/wp-con

**PREPORCESS IMAGES**

In [8]:
from skimage.transform import resize
from skimage.io import imread
import matplotlib.pyplot as plt
import os
import numpy as np

target = []
images = []     # matrix format
flat_data = []  # vector format

DATADIR = '/content/Mini_P'
CATEGORIES = ['acoustic guitar','electric guitar']
for category in CATEGORIES:
  class_num = CATEGORIES.index(category)
  path = os.path.join(DATADIR,category)
  for img in os.listdir(path):
    img_array = imread(os.path.join(path,img))
    img_resized = resize(img_array,(150,150,3))
    flat_data.append(img_resized.flatten())
    images.append(img_resized)
    target.append(class_num)

target = np.array(target)
images = np.array(images)
flat_data = np.array(flat_data)

**CREATE DATAFRAME**

In [9]:
import pandas as pd

df = pd.DataFrame(flat_data)
df['Target'] = target
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,67461,67462,67463,67464,67465,67466,67467,67468,67469,67470,67471,67472,67473,67474,67475,67476,67477,67478,67479,67480,67481,67482,67483,67484,67485,67486,67487,67488,67489,67490,67491,67492,67493,67494,67495,67496,67497,67498,67499,Target
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0
1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0
2,0.222105,0.171124,0.135830,0.222105,0.171124,0.135830,0.218183,0.167203,0.131908,0.220680,0.169699,0.134405,0.224601,0.173621,0.138327,0.194405,0.143425,0.108131,0.199647,0.148667,0.113373,0.197150,0.146170,0.110876,0.194654,0.155242,0.114797,0.162105,0.130732,0.087595,0.146170,0.111802,0.068665,0.147791,0.103033,0.065242,0.159359,0.108379,0.073085,0.165046,...,0.386186,0.269520,0.168183,0.375117,0.266667,0.171195,0.364155,0.258771,0.165580,0.347096,0.257897,0.164706,0.339556,0.257274,0.168752,0.321124,0.238771,0.167559,0.308039,0.231373,0.166079,0.297825,0.223316,0.156863,0.286863,0.212353,0.149233,0.270588,0.207843,0.149020,0.275935,0.213190,0.154366,0.274510,0.211765,0.154366,0.267165,0.204420,0.152941,0
3,0.470588,0.470588,0.470588,0.474510,0.474510,0.474510,0.478431,0.478431,0.478431,0.478431,0.478431,0.478431,0.482353,0.482353,0.482353,0.482353,0.482353,0.482353,0.486275,0.486275,0.486275,0.486275,0.486275,0.486275,0.490196,0.490196,0.490196,0.494118,0.494118,0.494118,0.494118,0.494118,0.494118,0.498039,0.498039,0.498039,0.498039,0.498039,0.498039,0.501961,...,0.498039,0.498039,0.498039,0.498039,0.498039,0.498039,0.494118,0.494118,0.494118,0.494118,0.494118,0.494118,0.490196,0.490196,0.490196,0.486275,0.486275,0.486275,0.486275,0.486275,0.486275,0.482353,0.482353,0.482353,0.482353,0.482353,0.482353,0.478431,0.478431,0.478431,0.475163,0.475163,0.475163,0.474510,0.474510,0.474510,0.470588,0.470588,0.470588,0
4,0.372827,0.310993,0.139349,0.416405,0.319925,0.149386,0.246758,0.210891,0.081817,0.210719,0.214641,0.084793,0.274510,0.278431,0.125490,0.248758,0.254902,0.098039,0.294027,0.301961,0.109804,0.319608,0.312359,0.120797,0.488771,0.376993,0.169020,0.779216,0.540000,0.334379,0.761150,0.549897,0.351241,0.668954,0.485582,0.286018,0.560719,0.435985,0.232261,0.470205,...,0.555033,0.376874,0.190614,0.427943,0.294458,0.161725,0.316694,0.230261,0.132183,0.204256,0.160954,0.114915,0.112444,0.107203,0.099320,0.078431,0.080392,0.086275,0.067856,0.074510,0.078431,0.070588,0.074510,0.082353,0.074902,0.078824,0.086667,0.082353,0.086275,0.094118,0.078431,0.078431,0.087266,0.078431,0.078431,0.086275,0.066389,0.070588,0.077242,0
...,...,...,...,...,...,...,...,...,...,...,...,.

**START BUILDING MODEL**

In [18]:
x = df.iloc[:,0:67500]
y = df.iloc[:,67500]

**SPLIT DATA**

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state = 0)

**USE GRID SEARCH CV TO FIND BEST PARAMETERS FOR K-NEAREST NEIGHBOURS**

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
z = np.arange(1,47)
param = {'n_neighbors':z}
model = KNeighborsClassifier()
model_grid = GridSearchCV(model,param)
model_grid.fit(x,y)

GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [13]:
model_grid.best_params_

{'n_neighbors': 4}

**FIT AND TRANSFORM THE TRAINING AND TESTING DATA**

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [15]:
model = KNeighborsClassifier(n_neighbors=4,metric='euclidean')
model.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

In [16]:
y_pred = model.predict(x_test)

In [17]:
# Evaluation of a model
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

0.68

#######################################################################################################
**RANDOM FOREST**

In [19]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(random_state = 1)
rf_model.fit(x_train,y_train)

y_pred2 = rf_model.predict(x_test)


In [20]:
for i in range(len(y_pred2)):
  y_pred2[i] = round(y_pred2[i])

**TEST ACCURACY**

In [21]:
# Evaluation of a model
from sklearn.metrics import accuracy_score
accuracy_score(y_pred2,y_test)

0.76

**ABSOLUTE MEAN OF ALL ERRORS**

In [22]:
from sklearn.metrics import mean_absolute_error
rf_test_mae = mean_absolute_error(y_pred2, y_test)
rf_test_mae

0.24